In [2]:
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt
import obspy
import util
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

In [10]:
root = '/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3/' #2018_2021_global_m5/'#'/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m5/'
eq_list = os.listdir(root)

In [31]:
#client = Client("IRIS")
#cat = client.get_events(starttime=UTCDateTime("2019-06-26"), endtime=UTCDateTime("2020-06-26"), minlongitude=-179, maxlongitude=-145, minlatitude=42, maxlatitude=71, minmagnitude=5, includearrivals=True, includeallmagnitudes = True)
cat = obspy.read_events('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3_catalog.xml') #2018_2021_global_m5_catalog.xml')

In [11]:
cat[0].magnitudes

[Magnitude(resource_id=ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=200228574"), mag=5.0, magnitude_type='Mww')]

In [36]:
len(cat)

15369

In [37]:
cat_with_data = cat.copy()
cat_with_data.clear()
for event in cat:
    eq_name = util.catEventToFileName(event)
    if os.path.isdir(root+eq_name) and os.path.isdir(root+eq_name+'/station_xml_files'):
        cat_with_data.extend([event])
        

In [43]:
mag_types = {}
for i in range(0, len(cat_with_data)):
    if float(cat_with_data[i].magnitudes[0].mag):
        key  = cat_with_data[i].magnitudes[0].magnitude_type
        key = key.lower()
        if key[0:2]=='mw':
            key = 'mw_all'


In [56]:
len(cat_with_data)

6077

In [55]:
types = {}
for e in cat_with_data:
    if len(e.magnitudes) > 1:
        mt = []
        print(e)
        for m in e.magnitudes:
            mt.append(m.magnitude_type[:2].lower())
        if 'mb' in mt and ('mw' in mt or 'ml' in mt):
            key = 'both'
        elif 'mw' in mt:
            key = 'mw'
        elif 'mb' in mt:
            key = 'mb'
        else:
            key = 'other'
        if key in mag_types.keys():
            types[key] = np.append(mag_types[key], e.magnitudes[0].mag)
        else:
            if e.magnitudes[0].mag != None:
                types[key] = np.array([e.magnitudes[0].mag])
        #types[key] = types.get(key, 0) + 1
        print(mt, '\n ===')

Event:	2019-01-31T13:09:53.370000Z |  +8.978,  -83.758 | 4.9 Mwr

	            resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?eventid=11001444")
	             event_type: 'earthquake'
	    preferred_origin_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=36901436")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=192340524")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 2 Elements
['mw', 'mb'] 
 ===
Event:	2019-01-31T03:52:50.297000Z | +61.928, -149.821 | 3.7 ML

	            resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?eventid=11001332")
	             event_type: 'earthquake'
	    preferred_origin_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=36896086")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:service.iris.edu/fdsn

In [57]:
len(types['both'])

108

In [ ]:
mag_types = {}
for i in range(0, len(cat_with_data)):
    if float(cat_with_data[i].magnitudes[0].mag):
        key  = cat_with_data[i].magnitudes[0].magnitude_type
        key = key.lower()
        if key[0:2]=='mw':
            key = 'mw_all'
        if key in mag_types.keys():
            mag_types[key] = np.append(mag_types[key], cat_with_data[i].magnitudes[0].mag)
        else:
            if cat_with_data[i].magnitudes[0].mag != None:
                mag_types[key] = np.array([cat_with_data[i].magnitudes[0].mag])

In [52]:
%matplotlib qt
mag_types = types.copy()
count = 0
for key in mag_types.keys():
    #plt.scatter(np.zeros(len(mag_types[key]))+count, mag_types[key])
    print(key, str(len(mag_types[key])))
    if len(mag_types[key])>0:
        plt.hist(mag_types[key], histtype = 'step', hatch = '.', label = key+str(len(mag_types[key])))
        #count += 1
plt.ylabel('n')
plt.xlabel('event magnitude')
plt.title('magnitude distribution by magnitude type')
plt.legend()

mb 7328
both 107
other 5643
mw 1499


In [ ]:
cat2 = cat_with_data.filter("magnitude >= 6.6")

In [ ]:
cat2

In [ ]:
cat2[0]